In [1]:
import pickle
import pandas as pd
import numpy as np
import tqdm

In [2]:
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression

from sklearn.multioutput import MultiOutputRegressor

In [3]:
!ls

analyse_states.ipynb	regressor_a_matrix.pkl	       states.csv
create_models.ipynb	regressor_gains.pkl	       videte_wheel.ipynb
params_a_matrixs_f.pkl	regressor_selection_gains.pkl  visualization.ipynb


In [4]:
df = pd.read_csv("states.csv")

df

,sec,nsec,frame_id,x1,x2,x3,x4,x5,x6,w1,...,w4,w5,w6,k1,k2,k3,k4,k5,k6,u
0,1600892364,11113280,robot,0.151688,-0.002232,0.017458,0.003468,-185.334625,0.0,0.1415,...,0.0,-183.822894,-0.0,-470.750654,-43.111772,1.323642,-6.585464,-3.130495,-5.313367,-0.002642
1,1600892364,15923534,robot,0.151735,0.001789,0.017458,0.003468,-185.334625,0.0,0.1415,...,0.0,-183.822894,-0.0,-470.750654,-43.111772,1.323642,-6.585464,-3.130495,-5.313367,0.012995
2,1600892364,20032963,robot,0.151735,0.001789,0.017124,-0.011715,-185.334625,0.0,0.1415,...,0.0,-183.822894,-0.0,-470.750654,-43.111772,1.323642,-6.585464,-3.130495,-5.313367,0.005031
3,1600892364,22898667,robot,0.151735,0.001789,0.017124,-0.011715,-185.334625,0.0,0.1415,...,0.0,-183.822894,-0.0,-470.750654,-43.111772,1.323642,-6.585464,-3.130495,-5.313367,0.005031
4,1600892364,28661242,robot,0.151789,0.000776,0.017124,-0.011715,-185.334625,0.0,0.1415,...,0.0,-183.822894,-0.0,-470.750654,-43.111772,1.323642,-6.585464,-3.130495,-5.313367,0.003587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30951,1600892455,391213348,robot,0.155436,-0.004590,-0.020278,0.002978,-180.897766,0.0,0.1415,...,0.0,-178.691134,-0.0,-470.750654,-43.111772,1.323642,-6.585464,-3.130495,-5.313367,-0.039909
30952,1600892455,393460227,robot,0.155436,-0.004590,-0.020278,0.002978,-180.897766,0.0,0.1415,...,0.0,-178.691134,-0.0,-470.750654,-43.111772,1.323642,-6.585464,-3.130495,-5.313367,-0.039909
30953,1600892455,395996733,robot,0.155407,0.000541,-0.020278,0.002978,-180.897766,0.0,0.1415,...,0.0,-178.691134,-0.0,-470.750654,-43.111772,1.323642,-6.585464,-3.130495,-5.313367,-0.023320
30954,1600892455,398442017,robot,0.155407,0.000541,-0.020278,0.002978,-180.897766,0.0,0.1415,...,0.0,-178.691134,-0.0,-470.750654,-43.111772,1.323642,-6.585464,-3.130495,-5.313367,-0.023320


In [5]:
states = df.loc[:, 'x1': 'x6']

states

,x1,x2,x3,x4,x5,x6
0,0.151688,-0.002232,0.017458,0.003468,-185.334625,0.0
1,0.151735,0.001789,0.017458,0.003468,-185.334625,0.0
2,0.151735,0.001789,0.017124,-0.011715,-185.334625,0.0
3,0.151735,0.001789,0.017124,-0.011715,-185.334625,0.0
4,0.151789,0.000776,0.017124,-0.011715,-185.334625,0.0
...,...,...,...,...,...,...
30951,0.155436,-0.004590,-0.020278,0.002978,-180.897766,0.0
30952,0.155436,-0.004590,-0.020278,0.002978,-180.897766,0.0
30953,0.155407,0.000541,-0.020278,0.002978,-180.897766,0.0
30954,0.155407,0.000541,-0.020278,0.002978,-180.897766,0.0


In [6]:
target = df.loc[:, 'w1': 'w6']

target

,w1,w2,w3,w4,w5,w6
0,0.1415,0.0,0.0,0.0,-183.822894,-0.0
1,0.1415,0.0,0.0,0.0,-183.822894,-0.0
2,0.1415,0.0,0.0,0.0,-183.822894,-0.0
3,0.1415,0.0,0.0,0.0,-183.822894,-0.0
4,0.1415,0.0,0.0,0.0,-183.822894,-0.0
...,...,...,...,...,...,...
30951,0.1415,0.0,0.0,0.0,-178.691134,-0.0
30952,0.1415,0.0,0.0,0.0,-178.691134,-0.0
30953,0.1415,0.0,0.0,0.0,-178.691134,-0.0
30954,0.1415,0.0,0.0,0.0,-178.691134,-0.0


In [7]:
control = df['u']

control

0       -0.002642
1        0.012995
2        0.005031
3        0.005031
4        0.003587
           ...   
30951   -0.039909
30952   -0.039909
30953   -0.023320
30954   -0.023320
30955   -0.023320
Name: u, Length: 30956, dtype: float64

In [8]:
pkl_filename = "regressor_a_matrix.pkl"
a_matrix = pd.read_pickle(pkl_filename)

In [9]:
pkl_filename = "regressor_gains.pkl"
gains = pd.read_pickle(pkl_filename)

In [10]:
A_open = a_matrix.predict([[0,0,0,0,0,0,0]]).reshape(6,6)
A_closed = a_matrix.predict([[0,0,0,0,0,0,200]]).reshape(6,6)

In [11]:
states.iloc[1]

x1      0.151735
x2      0.001789
x3      0.017458
x4      0.003468
x5   -185.334625
x6      0.000000
Name: 1, dtype: float64

In [12]:
B = [0.0, -134.30029546065, 0.0, 219.351159270877, 0.0, 511.649622338623]

In [13]:
control[0]

-0.002642095321789384

In [14]:
x_dot = A_open @ np.array(states.iloc[0]) + np.array(B) * control[0]

x_dot * 0.08

array([-1.78541669e-04,  1.48015026e+00,  2.77432000e-04, -2.05733290e-01,
        0.00000000e+00, -1.08146166e-01])

In [15]:
len(states)

30956

In [16]:
states.iloc[0] + x_dot * 0.08

x1      0.151509
x2      1.477918
x3      0.017735
x4     -0.202265
x5   -185.334625
x6     -0.108146
Name: 0, dtype: float64

In [17]:
prediction = []

for i in range(len(states)):
    prediction.append(states.iloc[i] + (A_open @ np.array(states.iloc[i]) + np.array(B) * control[i]) * 0.08)

prediction_open = pd.DataFrame(prediction)

prediction_open

,x1,x2,x3,x4,x5,x6
0,0.151509,1.477918,0.017735,-0.202265,-185.334625,-0.108146
1,0.151878,1.314387,0.017735,0.072130,-185.334625,0.531897
2,0.151878,1.399947,0.016187,-0.079752,-185.334625,0.205937
3,0.151878,1.399947,0.016187,-0.079752,-185.334625,0.205937
4,0.151851,1.414968,0.016187,-0.105089,-185.334625,0.146835
...,...,...,...,...,...,...
30951,0.155069,1.911830,-0.020039,-0.512238,-180.897766,-1.633553
30952,0.155069,1.911830,-0.020039,-0.512238,-180.897766,-1.633553
30953,0.155450,1.738450,-0.020039,-0.221137,-180.897766,-0.954542
30954,0.155450,1.738450,-0.020039,-0.221137,-180.897766,-0.954542


In [18]:
prediction = []

for i in range(len(states)):
    prediction.append(states.iloc[i] + (A_closed @ np.array(states.iloc[i]) + np.array(B) * control[i]) * 0.08)

prediction_closed = pd.DataFrame(prediction)

prediction_closed

,x1,x2,x3,x4,x5,x6
0,0.151509,1.477918,0.017735,-0.267537,-185.334625,-0.054176
1,0.151878,1.314387,0.017735,0.006858,-185.334625,0.585867
2,0.151878,1.399947,0.016187,0.140737,-185.334625,0.023623
3,0.151878,1.399947,0.016187,0.140737,-185.334625,0.023623
4,0.151851,1.414968,0.016187,0.115399,-185.334625,-0.035479
...,...,...,...,...,...,...
30951,0.155069,1.911830,-0.020039,-0.568290,-180.897766,-1.587205
30952,0.155069,1.911830,-0.020039,-0.568290,-180.897766,-1.587205
30953,0.155450,1.738450,-0.020039,-0.277190,-180.897766,-0.908195
30954,0.155450,1.738450,-0.020039,-0.277190,-180.897766,-0.908195


In [19]:
prediction_open - prediction_closed

,x1,x2,x3,x4,x5,x6
0,0.0,1.554312e-15,0.0,0.065271,0.0,-0.053971
1,0.0,1.998401e-15,0.0,0.065271,0.0,-0.053971
2,0.0,1.998401e-15,0.0,-0.220489,0.0,0.182315
3,0.0,1.998401e-15,0.0,-0.220489,0.0,0.182315
4,0.0,1.554312e-15,0.0,-0.220489,0.0,0.182315
...,...,...,...,...,...,...
30951,0.0,1.776357e-15,0.0,0.056052,0.0,-0.046348
30952,0.0,1.776357e-15,0.0,0.056052,0.0,-0.046348
30953,0.0,1.776357e-15,0.0,0.056052,0.0,-0.046348
30954,0.0,1.776357e-15,0.0,0.056052,0.0,-0.046348
